In [1]:
import pandas as pd
import numpy as np

## Getting the Data.

In [2]:
train_data=pd.read_csv("D:\Machine Learning\Twitter sentiments dataset\witter_train.csv")
test_data=pd.read_csv("D:\Machine Learning\Twitter sentiments dataset\witter_test.csv")

In [3]:
train_data.head()

,tweet_id,airline_sentiment,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,567900433542488064,negative,Southwest,NaN,ColeyGirouard,NaN,0,"@SouthwestAir I am scheduled for the morning, ...",NaN,2015-02-17 20:16:29 -0800,Washington D.C.,Atlantic Time (Canada)
1,569989168903819264,positive,Southwest,NaN,WalterFaddoul,NaN,0,@SouthwestAir seeing your workers time in and ...,NaN,2015-02-23 14:36:22 -0800,"Indianapolis, Indiana; USA",Central Time (US & Canada)
2,568089179520954368,positive,United,NaN,LocalKyle,NaN,0,@united Flew ORD to Miami and back and had gr...,NaN,2015-02-18 08:46:29 -0800,Illinois,Central Time (US & Canada)
3,568928195581513728,negative,Southwest,NaN,amccarthy19,NaN,0,@SouthwestAir @dultch97 that's horse radish 😤🐴,NaN,2015-02-20 16:20:26 -0800,NaN,Atlantic Time (Canada)
4,568594180014014464,negative,United,NaN,J_Okayy,NaN,0,@united so our flight into ORD was delayed bec...,NaN,2015-02-19 18:13:11 -0800,NaN,Eastern Time (US & Canada)


In [4]:
train_data.shape

(10980, 12)

In [5]:
X_train_text=((train_data[["text"]]).to_numpy())
X_train_text=np.reshape(X_train_text,10980)

In [6]:
Y_train=(train_data[["airline_sentiment"]]).to_numpy()
Y_train=np.reshape(Y_train,10980)

In [7]:
X_test_text=(test_data[["text"]]).to_numpy()
X_test_text=np.reshape(X_test_text,3660)

## Getting into right form:

In [37]:
def getting_words(d):
    doc=[]
    for i in range(0,len(d)):
        doc.append(word_tokenize(d[i]))
    return doc

In [48]:
X_train_words=getting_words(X_train_text)
X_test_words=getting_words(X_test_text)

In [10]:
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()

In [11]:
from nltk.corpus import wordnet
def get_simple_pos(tag):
    if tag.startswith("J"):
        return wordnet.ADJ
    elif tag.startswith("V"):
        return wordnet.VERB
    elif tag.startswith("N"):
        return wordnet.NOUN
    elif tag.startswith("R"):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [12]:
from nltk.corpus import stopword
import string
from nltk import pos_tag 
stops=set(stopwords.words("english"))
punctuations=list(string.punctuations)
stops.update(punctuations)

In [51]:
def clean_reviews(d):
    array_words=[]
    for i in range(0,len(d)):
        words=d[i]
        output_words=[]
        for w in words:
            if w.lower() not in stops:
                pos=pos_tag([w])
                clean_words=lemmatizer.lemmatize(w,pos=get_simple_pos(pos[0][1]))
                output_words.append(clean_words.lower())
        array_words.append(output_words)
    return array_words

In [52]:
X_train_words_clean=clean_reviews(X_train_words)
X_test_words_clean=clean_reviews(X_test_words)

In [62]:
all_words=[]
for doc in X_train_words_clean:
    all_words+=doc

In [66]:
import nltk
freq=nltk.FreqDist(all_words)
common=freq.most_common(2000)
features=[i[0] for i in common]

In [72]:
def get_feature_dict(d):
    out=[]
    for i in range(0,len(d)):
        words=d[i]
        current_features={}
        words_set=set(words)
        for w in features:
            current_features[w]=w in words_set
        out.append(current_features)
    return out

In [78]:
training_data_feat=get_feature_dict(X_train_words_clean)
testing_data=get_feature_dict(X_test_words_clean)

In [79]:
for i in range(len(training_data)):
    training_data=[(training_data_feat,Y_train[i])]

## Training and prediction:-

In [85]:
def join_words(d):
    out=[]
    for i in range(0,len(d)):
        out.append(" ".join(d[i]))
    return out

In [88]:
train_doc=join_words(X_train_words_clean)
test_doc=join_words(X_test_words_clean)

In [89]:
from sklearn.feature_extraction.text import CountVectorizer

In [186]:
count_vec=CountVectorizer(max_features=5000,ngram_range=(1,2))
x_train_features=count_vec.fit_transform(train_doc)

In [187]:
x_test_features=count_vec.transform(test_doc)

In [111]:
#from sklearn.svm import SVC
#svc=SVC()
#svc.fit(x_train_features,Y_train)

In [188]:
from sklearn.naive_bayes import MultinomialNB
clf=MultinomialNB()
clf.fit(x_train_features.todense(),Y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [144]:
#from sklearn.neighbors import KNeighborsClassifier
#clf=KNeighborsClassifier()
#clf.fit(x_train_features,Y_train)

In [182]:
#from sklearn.ensemble import RandomForestClassifier
#clf = RandomForestClassifier()
#clf.fit(x_train_features, Y_train)

C:\Users\ROHAN\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [189]:
pred_out=clf.predict(x_test_features.todense())

In [190]:
np.savetxt("twitter_sentiment.csv",pred_out,fmt="%s")